# Tutorial on collapsed Gibbs sampling

In [1]:
import numpy as np
from IPython.core.display import display, HTML, Image
import nbimporter
from utils import to_table


docs = [
    ['apple', 'ios', 'mac', 'book'],
    ['apple', 'mac', 'book', 'apple', 'store'],
    ['mac', 'book', 'ios', 'store'],
    ['banana', 'mango', 'fruit'],
    ['apple', 'fruit'],
    ['orange', 'strawberry', 'banana'],
    ['orange', 'mango', 'banana'],
    ['fruit', 'apple', 'mac', 'ios']
]
K = 2
words = list(set([x for y in docs for x in y]))
alpha = np.array([1]*K)
beta = np.array([1]*len(words))

importing Jupyter notebook from utils.ipynb


## Init counters of assignments
- $n_{d,k}$ number of words of the document $d$ assigned to $k$
- $n_{k,w}$ number of times (instances of) word $w$ is assigned to $k$
- $n_k$ number of word instances assignements to $k$
- $z$ array of assignments to topics for each of the words instances

In [2]:
N = sum([len(x) for x in docs])
z = np.zeros(N)
ndk = np.zeros((len(docs), K))
nkw = np.zeros((K, len(words)))
nk = np.zeros(K)

### Randomly initialize

In [3]:
instance = 0
for i, doc in enumerate(docs):
    for w in doc:
        t = np.random.choice(range(0,K))
        z[instance] = t
        nk[t] += 1
        ndk[i,t] += 1
        nkw[t, words.index(w)] += 1
        instance += 1

In [4]:
initial_assignment = to_table(z, ['topic'], [x for y in docs for x in y])
initial_ndk = to_table(ndk, range(0,len(docs)), range(0,K))
initial_nkw = to_table(nkw, range(0,K), words)

In [5]:
display(HTML(initial_assignment))
display(HTML(initial_ndk))
display(HTML(initial_nkw))

,apple,ios,mac,book,apple,mac,book,apple,store,mac,book,ios,store,banana,mango,fruit,apple,fruit,orange,strawberry,banana,orange,mango,banana,fruit,apple,mac,ios
topic,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


,0,1
0,1.0,3.0
1,3.0,2.0
2,1.0,3.0
3,2.0,1.0
4,1.0,1.0
5,2.0,1.0
6,2.0,1.0
7,4.0,0.0


,apple,ios,fruit,mango,strawberry,mac,book,orange,banana,store
0,3.0,1.0,1.0,2.0,1.0,1.0,3.0,0.0,3.0,1.0
1,2.0,2.0,2.0,0.0,0.0,3.0,0.0,2.0,0.0,1.0


## Likelihood

In [6]:
def theta(ndk, alpha, d_i, k_i):
    return (ndk[d_i, k_i] + alpha[k_i]) / (np.sum(ndk[d_i,:]) + alpha[k_i])

def phi(nkw, beta, w_i, k_i):
    return (nkw[k_i, w_i] + beta[w_i]) / (np.sum(nkw[k_i,:] + beta))

In [7]:
print phi(ndk, alpha, 1, 0)

0.666666666667


# Gibbs Sampling

In [8]:
def gibbs(docs, words, topics, ndk, nkw, nk, alpha, beta, iterations=10):
    for it in range(0, iterations):
        w_i = 0
        for doc_i, doc in enumerate(docs):
            for w in doc:
                word = [x for y in docs for x in y][w_i]
                topic = int(topics[w_i])
                # Remove current assignment
                ndk[doc_i, topic] -= 1
                nkw[topic, words.index(word)] -= 1
                nk[topic] -= 1
                p_z = np.zeros(len(alpha))
                for k_i in range(0, len(alpha)):
                    p_z[k_i] = theta(ndk, alpha, doc_i, k_i) * phi(nkw, beta, words.index(word), k_i)
                p_z = p_z / np.sum(p_z)
                # Sample from p_z
                new_topic = np.random.choice(len(p_z), 1, p=p_z)[0]
                # Update
                topics[w_i] = new_topic
                ndk[doc_i, new_topic] += 1
                nkw[new_topic, words.index(word)] += 1
                nk[new_topic] += 1
                w_i += 1

In [9]:
gibbs(docs, words, z, ndk, nkw, nk, alpha, beta, iterations=100)

In [10]:
final_assignment = to_table(z, ['topic'], [x for y in docs for x in y])
final_ndk = to_table(ndk, range(0,len(docs)), range(0,K))
final_nkw = to_table(nkw, range(0,K), words)

In [11]:
display(HTML(initial_assignment))
display(HTML(final_assignment))

,apple,ios,mac,book,apple,mac,book,apple,store,mac,book,ios,store,banana,mango,fruit,apple,fruit,orange,strawberry,banana,orange,mango,banana,fruit,apple,mac,ios
topic,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


,apple,ios,mac,book,apple,mac,book,apple,store,mac,book,ios,store,banana,mango,fruit,apple,fruit,orange,strawberry,banana,orange,mango,banana,fruit,apple,mac,ios
topic,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0


In [12]:
display(HTML(initial_ndk))
display(HTML(final_ndk))

,0,1
0,1.0,3.0
1,3.0,2.0
2,1.0,3.0
3,2.0,1.0
4,1.0,1.0
5,2.0,1.0
6,2.0,1.0
7,4.0,0.0


,0,1
0,3.0,1.0
1,4.0,1.0
2,2.0,2.0
3,0.0,3.0
4,0.0,2.0
5,3.0,0.0
6,2.0,1.0
7,0.0,4.0


In [13]:
display(HTML(initial_nkw))
display(HTML(final_nkw))

,apple,ios,fruit,mango,strawberry,mac,book,orange,banana,store
0,3.0,1.0,1.0,2.0,1.0,1.0,3.0,0.0,3.0,1.0
1,2.0,2.0,2.0,0.0,0.0,3.0,0.0,2.0,0.0,1.0


,apple,ios,fruit,mango,strawberry,mac,book,orange,banana,store
0,1.0,1.0,0.0,1.0,1.0,3.0,3.0,1.0,2.0,1.0
1,4.0,2.0,3.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0


# Sklearn implementation

In [14]:
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter


M = np.zeros((len(words), len(docs)))
for i, doc in enumerate(docs):
    for k, w in Counter(doc).most_common():
        M[words.index(k), i] = w
LDA = LatentDirichletAllocation(n_topics=2, learning_method='batch', max_iter=100).fit(M.T)

/usr/local/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


In [15]:
A = LDA.components_

Topic word distribution. components_[i, j] represents word j in topic i.

In [16]:
display(HTML(to_table(A, range(0,K), words)))

,apple,ios,fruit,mango,strawberry,mac,book,orange,banana,store
0,0.513,0.51,1.586,2.495,1.496,0.509,0.509,2.496,3.496,0.508
1,5.487,3.49,2.414,0.505,0.504,4.491,3.491,0.504,0.504,2.492


In [17]:
Terms = np.zeros((len(words), K))
Docs = np.zeros((len(docs), K))

for w_i, w in enumerate(words):
    Terms[w_i] = A.T[w_i] / np.sum(A.T[w_i])

for d_i, d in enumerate(docs):
    dM = np.zeros((len(d), K))
    for j, w in enumerate(d):
        dM[j] = Terms[words.index(w)]
    Docs[d_i] = np.sum(dM, axis=0) / np.array([len(d)]*K)

In [18]:
display(HTML(to_table(Terms, words, range(0,K))))
display(HTML(to_table(Docs, range(0, len(docs)), range(0,K))))

,0,1
apple,0.086,0.914
ios,0.128,0.872
fruit,0.397,0.603
mango,0.832,0.168
strawberry,0.748,0.252
mac,0.102,0.898
book,0.127,0.873
orange,0.832,0.168
banana,0.874,0.126
store,0.169,0.831


,0,1
0,0.111,0.889
1,0.114,0.886
2,0.131,0.869
3,0.701,0.299
4,0.241,0.759
5,0.818,0.182
6,0.846,0.154
7,0.178,0.822


In [19]:
sigma_w, sigma_d = 4, 3
for t_k in range(0, K):
    print 'topic', t_k
    top_w = sorted([(i, wk) for i, wk in enumerate(Terms.T[t_k])], key=lambda y: -y[1])[:sigma_w]
    top_d = sorted([(i, dk) for i, dk in enumerate(Docs.T[t_k])], key=lambda y: -y[1])[:sigma_d]
    print [words[x[0]] for x in top_w]
    print [docs[x[0]] for x in top_d]
    print ""

topic 0
['banana', 'orange', 'mango', 'strawberry']
[['orange', 'mango', 'banana'], ['orange', 'strawberry', 'banana'], ['banana', 'mango', 'fruit']]

topic 1
['apple', 'mac', 'book', 'ios']
[['apple', 'ios', 'mac', 'book'], ['apple', 'mac', 'book', 'apple', 'store'], ['mac', 'book', 'ios', 'store']]

